In [2]:
import pandas as pd
import os
import hashlib
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
!pip3 install pyproj
from pyproj import Proj, transform
inProj = Proj(init='epsg:28992')
outProj = Proj(init='epsg:4326')

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.3 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [224]:
data_file_path="~/SageMaker/Rain_Estimation_Using_Loosely_Matched_Stations/Gauges_20110609_20110813.csv"
metadata_file_path = "~/SageMaker/Rain_Estimation_Using_Loosely_Matched_Stations/dutch_gauge_coo.csv"

In [225]:
df = pd.read_csv(data_file_path, sep=",", index_col=False)
metadata_df  = pd.read_csv(metadata_file_path, sep=",", index_col=False)

In [226]:
df[df.RH=="-1"]="0"
df.drop(df[df.RH==" "].index,inplace=True)

In [227]:
stations=df.STN.unique()
stations = stations[stations != "0"]
len(stations)

32

In [228]:
stations

array([210, 235, 240, 249, 251, 257, 260, 267, 269, 270, 273, 275, 277,
       278, 279, 280, 283, 286, 290, 310, 319, 323, 330, 344, 348, 350,
       356, 370, 375, 377, 380, 391], dtype=object)

In [229]:
def convert_from_dutch_coo_to_longitude_latitude(x1,y1):
    return transform(inProj,outProj,x1*1000,y1*1000)

In [230]:
metadata_df=metadata_df.join(metadata_df.apply(lambda x: pd.Series({"long": convert_from_dutch_coo_to_longitude_latitude(x.POS_X, x.POS_Y)[0],
                                       "lat": convert_from_dutch_coo_to_longitude_latitude(x.POS_X, x.POS_Y)[1]}), axis=1))

/tmp/ipykernel_11733/1893588703.py:2: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return transform(inProj,outProj,x1*1000,y1*1000)


In [231]:
metadata_df.head()

,STN,STARTT,STOPT,LOCATIE,HOOGTE,POS_X,POS_Y,POS_NB,POS_OL,long,lat
0,201,20090313,99991231,platform D15-FA-1,42.7,-4.5,707.1,54.3,2.9,2.936152,54.323898
1,203,20091109,99991231,platform P11-B,41.8,15.7,487.6,52.4,3.3,3.342154,52.358603
2,204,20061206,99991231,platform K14-FA-1C,41.8,37.6,588.3,53.3,3.6,3.627506,53.268329
3,205,20090313,99991231,platform A12-CPP,48.4,55.0,825.1,55.4,3.8,3.809936,55.397968
4,206,20061206,99991231,platform F16-A,43.4,65.1,682.0,54.1,4.0,4.012695,54.115259


In [232]:
def convert_date(date):
    yyyy=date[:4]
    mm=date[4:6]
    dd=date[6:8]
    return f"{dd}-{mm}-{yyyy}"

def convert_time(time):
    if time=="24":
        hh="00"
    else:
        hh=f"0{time}" if len(time)==1 else f"{time}"
    
    mm="00"
    ss="00"
    
    return f"{hh}:{mm}:{ss}"
    

In [233]:
stations

array([210, 235, 240, 249, 251, 257, 260, 267, 269, 270, 273, 275, 277,
       278, 279, 280, 283, 286, 290, 310, 319, 323, 330, 344, 348, 350,
       356, 370, 375, 377, 380, 391], dtype=object)

In [238]:
df["Time"]=df["YYYYMMDD"].astype(str).apply(convert_date)+" "+df["H"].astype(str).apply(convert_time)
df.RH=df.RH.astype(int)//10
df.rename(columns={"RH": "RainAmout[mm/h]"}, inplace=True)

In [239]:
# for station in stations:
#     df[df.STN==station].apply(lambda x: pd.Series({"Time":convert_datetime_to_time(x.YYYYMMDD,x.H)}
    

In [240]:
columns=["Time","RainAmout[mm/h]"]
root="~/SageMaker/Rain_Estimation_Using_Loosely_Matched_Stations/CellEnMon/datasets/ims/09062011_13082011/raw"
for station in stations:
    d=df[df.STN==station]
    try:
        long=metadata_df[metadata_df.STN==station].long.iloc[0]
        lat=metadata_df[metadata_df.STN==station].lat.iloc[0]
        d.to_csv(f'{root}/{d.STN.iloc[0]}_{long:.3f}_{lat:.3f}.csv',index=False, columns=columns)
    except IndexError:
        print(f"This station does not exist:{station}")

This station does not exist:210


In [180]:
metadata_df[metadata_df.STN==station].long.iloc[0]

4.788843987616406

In [3]:
raw_data = {'company': ['comp1', 'comp1', 'comp1', 'comp1', 'comp2', 'comp2', 'comp2', 'comp2', 'comp3', 'comp3', 'comp3', 'comp3'], 
        'region': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'], 
        'name': ['John', 'Jake', 'Alice', 'Mathew', 'Mark', 'Jacon', 'Ryan', 'Sone', 'Steve', 'Rooke', 'Rani', 'Alice'], 
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}
df1 = pd.DataFrame(raw_data, columns = ['company', 'region', 'name', 'preTestScore'])
df1

,company,region,name,preTestScore
0,comp1,1st,John,4
1,comp1,1st,Jake,24
2,comp1,2nd,Alice,31
3,comp1,2nd,Mathew,2
4,comp2,1st,Mark,3
5,comp2,1st,Jacon,4
6,comp2,2nd,Ryan,24
7,comp2,2nd,Sone,31
8,comp3,1st,Steve,2
9,comp3,1st,Rooke,3


In [11]:
raw_data = {'company': [ 'comp1'], 
        'region': [ '2nd'], 
        'name': [ 'Alice' ], 
        'status': [ 'great'],
       'preTestScore': [22]}
df2 = pd.DataFrame(raw_data, columns = ['company', 'region', 'name', 'status', 'preTestScore'])
df2

,company,region,name,status,preTestScore
0,comp1,2nd,Alice,great,22


In [13]:
df_merged=pd.merge(df1, df2, how='inner', on=['company'])

In [15]:
sum(df_merged["preTestScore_x"]-df_merged["preTestScore_y"])

-27

In [16]:
a={1:2,4:5}

In [19]:
{**a,**{5:6}}

{1: 2, 4: 5, 5: 6}